In [1]:
import tensorflow as tf
from dataset_tf import VoxDataset, DataGenerator
from tqdm import tqdm
from sklearn.model_selection import train_test_split
import pandas as pd
# from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping


In [2]:
dataset = VoxDataset()

In [3]:
ids = dataset.get_ids()

107640


In [4]:
len(ids)

5382

In [5]:
len(dataset)

107640

In [6]:
DATASET_SIZE = len(ids)

In [7]:
train_size = int(0.6 * DATASET_SIZE)
val_size = int(0.2 * DATASET_SIZE)
test_size = DATASET_SIZE - train_size - val_size

In [8]:
print(train_size)
print(test_size)
print(val_size)

3229
1077
1076


In [9]:
np.random.shuffle(ids)

# Split the indices into training, validation, and test sets
train_indices = ids[:train_size]
val_indices = ids[train_size:train_size + val_size]
test_indices = ids[train_size + val_size:]

In [10]:
len(train_indices)

3229

In [11]:
train = VoxDataset(ids=train_indices)
test = VoxDataset(ids=test_indices)
val = VoxDataset(ids=val_indices)

In [12]:
type(train_indices)

list

In [13]:
print(len(train))
print(len(test))
print(len(val))

64580
21540
21520


In [14]:
train[0][1]

2024-03-21 21:37:00.006216: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-03-21 21:37:00.006235: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-21 21:37:00.006242: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-21 21:37:00.006271: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-21 21:37:00.006286: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


<tf.Tensor: shape=(15069,), dtype=float32, numpy=
array([ 0.23342162, -0.03763863, -0.18366627, ..., -0.11938662,
        0.07832987,  0.03457367], dtype=float32)>

In [15]:
import tensorflow as tf

def AutoEncoder():
    # Define input layer
    input_layer = tf.keras.layers.Input(shape=(384,))
    
    # Encoder layers
    encoder = tf.keras.layers.Dense(192, activation='tanh')(input_layer)
    encoder = tf.keras.layers.Dense(96, activation='tanh')(encoder)
    

    attention = tf.keras.layers.Attention()([encoder, encoder])
    weighted_bottleneck = tf.keras.layers.multiply([attention, encoder])

    attention = tf.keras.layers.Attention()([weighted_bottleneck, weighted_bottleneck])
    weighted_bottleneck = tf.keras.layers.multiply([attention, weighted_bottleneck])
    
    attention = tf.keras.layers.Attention()([weighted_bottleneck, weighted_bottleneck])
    weighted_bottleneck = tf.keras.layers.multiply([attention, weighted_bottleneck])

    attention = tf.keras.layers.Attention()([weighted_bottleneck, weighted_bottleneck])
    weighted_bottleneck = tf.keras.layers.multiply([attention, weighted_bottleneck])

    
    # Bottleneck Decoders 
    decoder = tf.keras.layers.Dense(96, activation='tanh')(weighted_bottleneck)
    decoder = tf.keras.layers.Dense(192, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(384, activation='tanh')(decoder)
    # Decoder layers
    decoder = tf.keras.layers.Dense(500, activation='tanh')(encoder)
    decoder = tf.keras.layers.Dense(768, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(1000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(1536, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(2000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(3072, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(5000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(6144, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(8000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(10000, activation='tanh')(decoder)
    decoder = tf.keras.layers.Dense(12288, activation='tanh')(decoder)
    decoder_output = tf.keras.layers.Dense(15069, activation='tanh')(decoder)
    
    # Define the model
    model = tf.keras.models.Model(inputs=input_layer, outputs=decoder_output)
    
    return model



In [16]:
num_epochs = 100
batch= 64

In [17]:
train = DataGenerator(train, batch_size=batch)
val = DataGenerator(val, batch_size=batch)
test = DataGenerator(test, batch_size=1)

In [18]:
model = AutoEncoder()


In [19]:
loss_fn = tf.keras.losses.MeanSquaredError()

# Define the optimizer
optimizer = tf.keras.optimizers.legacy.Adam(learning_rate=0.00001)

# Compile the model
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

early_stopping = EarlyStopping(monitor='val_loss', patience=3, verbose=1)
# Train the model using model.fit
model.fit(train, epochs=num_epochs, batch_size=batch, validation_data=val, 
          callbacks=[early_stopping]
          )

Epoch 1/100


2024-03-21 21:37:02.953405: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


1010/1010 [==============================] - 391s 385ms/step - loss: 7.2603e-04 - accuracy: 0.3600 - val_loss: 1.3358e-04 - val_accuracy: 0.3986
Epoch 2/100
1010/1010 [==============================] - 387s 383ms/step - loss: 1.2415e-04 - accuracy: 0.5652 - val_loss: 1.0903e-04 - val_accuracy: 0.8161
Epoch 3/100
1010/1010 [==============================] - 411s 407ms/step - loss: 1.0532e-04 - accuracy: 0.6337 - val_loss: 1.1562e-04 - val_accuracy: 0.5388
Epoch 4/100
1010/1010 [==============================] - 383s 378ms/step - loss: 9.1568e-05 - accuracy: 0.6453 - val_loss: 7.3041e-05 - val_accuracy: 0.9803
Epoch 5/100
1010/1010 [==============================] - 367s 362ms/step - loss: 8.0430e-05 - accuracy: 0.6440 - val_loss: 6.6457e-05 - val_accuracy: 0.4197
Epoch 6/100
1010/1010 [==============================] - 382s 378ms/step - loss: 7.3007e-05 - accuracy: 0.6264 - val_loss: 6.5249e-05 - val_accuracy: 0.0461
Epoch 7/100
1010/1010 [==============================] - 383s 379ms/st

KeyboardInterrupt: 

In [ ]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 384)]             0         
                                                                 
 dense (Dense)               (None, 192)               73920     
                                                                 
 dense_1 (Dense)             (None, 96)                18528     
                                                                 
 dense_5 (Dense)             (None, 500)               48500     
                                                                 
 dense_6 (Dense)             (None, 768)               384768    
                                                                 
 dense_7 (Dense)             (None, 1000)              769000    
                                                                 
 dense_8 (Dense)             (None, 1536)              153753

In [ ]:
loss, acc = model.evaluate(test)

21540/21540 [==============================] - 439s 20ms/step - loss: 5.3700e-05 - accuracy: 1.0000


In [ ]:
print(f"Accuracy: {acc*100:.4f}%\nLoss: {loss:.4f}")

Accuracy: 99.9954%
Loss: 0.0001


In [ ]:
model.save('./scaled_-1_1_batch_64_act_relu_with_enc_att_4l.h5', save_format='h5')

/Users/naweenk/miniforge3/envs/tf/lib/python3.10/site-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
# model = tf.keras.models.load_model("scaled_-1_1_batch_8_act_tanh_with_enc_att.h5")


In [ ]:
y_pred = []

In [ ]:
for i in tqdm(test):
    pred = (model.predict(i[0], verbose=0)*279.79984)
    y_pred.append(VoxDataset.to_mesh_points(pred))



100%|██████████| 21540/21540 [15:48<00:00, 22.70it/s]


In [ ]:
type(y_pred[0])

numpy.ndarray

In [ ]:
import open3d as o3d
import numpy as np

for index, i in enumerate(tqdm(y_pred)):
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(i)
    # o3d.visualization.draw_geometries([pcd])
    o3d.io.write_point_cloud(f"./Test/data_{index}.ply", pcd)

100%|██████████| 21540/21540 [01:07<00:00, 320.03it/s]


In [ ]:
# import numpy as np
# from scipy.spatial import Delaunay
# import trimesh

# points = y_pred[0]

# pcd = o3d.geometry.PointCloud()
# pcd.points = o3d.utility.Vector3dVector(points)

# # Estimate normals for the point cloud
# pcd.estimate_normals()

# # Create a surface mesh from the point cloud using Poisson reconstruction
# mesh, densities = o3d.geometry.TriangleMesh.create_from_point_cloud_poisson(pcd, depth=9)

# # Visualize the mesh (optional)
# o3d.visualization.draw_geometries([mesh])